In [27]:
import requests
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso


def get_bitcoin_data():
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    params = {
        'vs_currency': 'usd',
        'days': '365',
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

def prepare_data():
    data = get_bitcoin_data()
    prices = data['prices']


    df = pd.DataFrame(prices, columns=['timestamp', 'price'])


    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    return df

df = prepare_data()
print(df.head())


                   price
timestamp               
2023-09-16  26634.631025
2023-09-17  26557.768692
2023-09-18  26520.988255
2023-09-19  26741.461111
2023-09-20  27219.296875


In [8]:
#Pré processamento dos dados. Pegando do dia anterior, dos ultimos 7 dias e a média dos últimos 7 dias
def create_features(df):
    # Criar lags de 1 e 7 dias
    df['lag_1'] = df['price'].shift(1)
    df['lag_7'] = df['price'].shift(7)


    df['ma_7'] = df['price'].rolling(window=7).mean()

    df = df.dropna()

    return df

df_features = create_features(df)
print(df_features.head())


                   price         lag_1         lag_7          ma_7
timestamp                                                         
2023-09-23  26572.038112  26561.133454  26634.631025  26755.504706
2023-09-24  26573.923480  26572.038112  26557.768692  26757.812533
2023-09-25  26249.562898  26573.923480  26520.988255  26719.037482
2023-09-26  26298.634678  26249.562898  26741.461111  26655.776563
2023-09-27  26204.757591  26298.634678  27219.296875  26510.842380


In [9]:
X = df_features[['lag_1', 'lag_7', 'ma_7']]
y = df_features['price']


In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


Par RIDGE LASSO

In [46]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

alphas = [0.01, 0.1, 1.0, 10.0]
solvers = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg']  
max_iters = [100, 500, 1000]  

ridge_metrics = {}
lasso_metrics = {}

for alpha in alphas:
    for solver in solvers:
        ridge_model = Ridge(alpha=alpha, solver=solver)
        ridge_model.fit(X_train, y_train)
        y_pred = ridge_model.predict(X_test)
        ridge_metrics[(alpha, solver)] = {
            'MSE': mean_squared_error(y_test, y_pred),
            'R²': r2_score(y_test, y_pred)
        }

for alpha in alphas:
    for max_iter in max_iters:
        lasso_model = Lasso(alpha=alpha, max_iter=max_iter)
        lasso_model.fit(X_train, y_train)
        y_pred = lasso_model.predict(X_test)
        lasso_metrics[(alpha, max_iter)] = {
            'MSE': mean_squared_error(y_test, y_pred),
            'R²': r2_score(y_test, y_pred)
        }

ridge_metrics_df = pd.DataFrame(ridge_metrics).T
lasso_metrics_df = pd.DataFrame(lasso_metrics).T

print("Métricas da Regressão Ridge:")
print(ridge_metrics_df)

print("\nMétricas da Regressão Lasso:")
print(lasso_metrics_df)

best_params_ridge = ridge_metrics_df['MSE'].idxmin()
best_params_lasso = lasso_metrics_df['MSE'].idxmin()

print("\nMelhores Parâmetros da Regressão Ridge:")
print(best_params_ridge)

print("\nMelhores Parâmetros da Regressão Lasso:")
print(best_params_lasso)


Métricas da Regressão Ridge:
                          MSE        R²
0.01  auto       2.446619e+06  0.986019
      svd        2.446619e+06  0.986019
      cholesky   2.446619e+06  0.986019
      lsqr       2.446619e+06  0.986019
      sparse_cg  2.446619e+06  0.986019
0.10  auto       2.446619e+06  0.986019
      svd        2.446619e+06  0.986019
      cholesky   2.446619e+06  0.986019
      lsqr       2.446619e+06  0.986019
      sparse_cg  2.446619e+06  0.986019
1.00  auto       2.446619e+06  0.986019
      svd        2.446619e+06  0.986019
      cholesky   2.446619e+06  0.986019
      lsqr       2.446619e+06  0.986019
      sparse_cg  2.446619e+06  0.986019
10.00 auto       2.446619e+06  0.986019
      svd        2.446619e+06  0.986019
      cholesky   2.446619e+06  0.986019
      lsqr       2.446619e+06  0.986019
      sparse_cg  2.446619e+06  0.986019

Métricas da Regressão Lasso:
                     MSE        R²
0.01  100   2.369525e+06  0.986459
      500   2.431496e+06  0.986

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.646e+08, tolerance: 4.801e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.547e+08, tolerance: 4.801e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-p

PAR GradientBoosting

In [47]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

n_estimators = [50, 100, 200]
max_depth = [3, 5, 7]
learning_rates = [0.01, 0.1, 0.2]
min_samples_split = [2, 5, 10]

gb_metrics = {}

for n in n_estimators:
    for depth in max_depth:
        for lr in learning_rates:
            for min_split in min_samples_split:
                gb_model = GradientBoostingRegressor(
                    n_estimators=n,
                    max_depth=depth,
                    learning_rate=lr,
                    min_samples_split=min_split
                )
                gb_model.fit(X_train, y_train)
                y_pred = gb_model.predict(X_test)
                gb_metrics[(n, depth, lr, min_split)] = {
                    'MSE': mean_squared_error(y_test, y_pred),
                    'R²': r2_score(y_test, y_pred)
                }

gb_metrics_df = pd.DataFrame(gb_metrics).T
print("Métricas do Gradient Boosting:")
print(gb_metrics_df)

best_params_gb = gb_metrics_df['MSE'].idxmin()
print("\nMelhores Parâmetros:")
print(best_params_gb)


Métricas do Gradient Boosting:
                        MSE        R²
50  3 0.01 2   6.724901e+07  0.615709
           5   6.724408e+07  0.615737
           10  6.722158e+07  0.615865
      0.10 2   3.325504e+06  0.980997
           5   3.333973e+06  0.980948
...                     ...       ...
200 7 0.10 5   3.659864e+06  0.979086
           10  3.623319e+06  0.979295
      0.20 2   3.833777e+06  0.978092
           5   3.912725e+06  0.977641
           10  3.444201e+06  0.980318

[81 rows x 2 columns]

Melhores Parâmetros:
(np.int64(100), np.int64(3), np.float64(0.1), np.int64(10))


xgboost

In [48]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

n_estimators = [50, 100, 200]
learning_rates = [0.01, 0.1, 0.2]
max_depth = [3, 5, 7]
subsample = [0.8, 0.9, 1.0]

xgb_metrics = {}

for n in n_estimators:
    for lr in learning_rates:
        for depth in max_depth:
            for subs in subsample:
                xgb_model = xgb.XGBRegressor(
                    objective='reg:squarederror',
                    n_estimators=n,
                    learning_rate=lr,
                    max_depth=depth,
                    subsample=subs
                )
                xgb_model.fit(X_train, y_train)
                y_pred = xgb_model.predict(X_test)
                xgb_metrics[(n, lr, depth, subs)] = {
                    'MSE': mean_squared_error(y_test, y_pred),
                    'R²': r2_score(y_test, y_pred)
                }

xgb_metrics_df = pd.DataFrame(xgb_metrics).T
print("Métricas do XGBoost:")
print(xgb_metrics_df)

best_params_xgb = xgb_metrics_df['MSE'].idxmin()
print("\nMelhores Parâmetros:")
print(best_params_xgb)


Métricas do XGBoost:
                         MSE        R²
50  0.01 3 0.8  7.058787e+07  0.596629
           0.9  7.021133e+07  0.598781
           1.0  6.990041e+07  0.600557
         5 0.8  7.029831e+07  0.598284
           0.9  6.983088e+07  0.600955
...                      ...       ...
200 0.20 5 0.9  3.702214e+06  0.978844
           1.0  3.791393e+06  0.978334
         7 0.8  3.961905e+06  0.977360
           0.9  3.547905e+06  0.979726
           1.0  3.651771e+06  0.979132

[81 rows x 2 columns]

Melhores Parâmetros:
(np.int64(50), np.float64(0.1), np.int64(3), np.float64(0.9))


MLPRegressor

In [53]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

hidden_layer_sizes = [(50,), (100,), (100, 50), (100, 100), (50, 50, 50)]
activation = ['relu', 'tanh', 'logistic']
alpha = [0.0001, 0.001, 0.01]
solver = ['lbfgs', 'sgd', 'adam']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

nn_metrics = {}

for layers in hidden_layer_sizes:
    for act in activation:
        for alpha_val in alpha:
            for solver_method in solver:
                try:
                    nn_model = MLPRegressor(
                        hidden_layer_sizes=layers,
                        activation=act,
                        alpha=alpha_val,
                        solver=solver_method,
                        max_iter=1000,
                        verbose=True,
                        learning_rate_init=0.01
                    )
                    nn_model.fit(X_train_scaled, y_train)
                    y_pred = nn_model.predict(X_test_scaled)
                    nn_metrics[(layers, act, alpha_val, solver_method)] = {
                        'MSE': mean_squared_error(y_test, y_pred),
                        'R²': r2_score(y_test, y_pred)
                    }
                except Exception as e:
                    print(f"Error with parameters {layers}, {act}, {alpha_val}, {solver_method}: {e}")

nn_metrics_df = pd.DataFrame(nn_metrics).T
print("Métricas do MLP Regressor:")
print(nn_metrics_df)

best_params_nn = nn_metrics_df['MSE'].idxmin()
print("\nMelhores Parâmetros:")
print(best_params_nn)


Iteration 1, loss = 64170039417.18319702
Iteration 2, loss = 2657587310384388957792234850322102396816087350355616737382843027844616945664.00000000
Iteration 3, loss = inf
Iteration 4, loss = nan
Iteration 5, loss = nan
Iteration 6, loss = nan
Iteration 7, loss = nan
Iteration 8, loss = nan
Iteration 9, loss = nan
Iteration 10, loss = nan
Iteration 11, loss = nan
Iteration 12, loss = nan
Iteration 13, loss = nan
Iteration 14, loss = nan
Iteration 15, loss = nan
Iteration 16, loss = nan
Iteration 17, loss = nan
Iteration 18, loss = nan
Iteration 19, loss = nan
Iteration 20, loss = nan
Iteration 21, loss = nan
Iteration 22, loss = nan
Iteration 23, loss = nan
Iteration 24, loss = nan
Iteration 25, loss = nan
Iteration 26, loss = nan
Iteration 27, loss = nan
Iteration 28, loss = nan
Iteration 29, loss = nan
Iteration 30, loss = nan
Iteration 31, loss = nan
Iteration 32, loss = nan
Iteration 33, loss = nan
Iteration 34, loss = nan
Iteration 35, loss = nan
Iteration 36, loss = nan
Iteration 

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:205: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:205: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:174: RuntimeWarning: invalid value encountered in add
  activations[i + 1] += self.intercepts_[i]
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations

Iteration 244, loss = 1324117428.79830861
Iteration 245, loss = 1322406842.18155456
Iteration 246, loss = 1320696702.82875586
Iteration 247, loss = 1318996995.45208812
Iteration 248, loss = 1317290109.32859731
Iteration 249, loss = 1315584453.08222413
Iteration 250, loss = 1313839774.12186790
Iteration 251, loss = 1312148176.67342544
Iteration 252, loss = 1310416622.10070491
Iteration 253, loss = 1308668826.34212375
Iteration 254, loss = 1306973857.34713387
Iteration 255, loss = 1305217479.78652167
Iteration 256, loss = 1303489001.67490292
Iteration 257, loss = 1301761696.76889515
Iteration 258, loss = 1300006695.24689865
Iteration 259, loss = 1298250882.13687396
Iteration 260, loss = 1296485467.63606715
Iteration 261, loss = 1294731491.25560188
Iteration 262, loss = 1292978593.13598084
Iteration 263, loss = 1291191213.58845043
Iteration 264, loss = 1289417096.41889524
Iteration 265, loss = 1287652849.74672532
Iteration 266, loss = 1285879124.21744227
Iteration 267, loss = 1284095392.0

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:205: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:205: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:174: RuntimeWarning: i

Iteration 203, loss = nan
Iteration 204, loss = nan
Iteration 205, loss = nan
Iteration 206, loss = nan
Iteration 207, loss = nan
Iteration 208, loss = nan
Iteration 209, loss = nan
Iteration 210, loss = nan
Iteration 211, loss = nan
Iteration 212, loss = nan
Iteration 213, loss = nan
Iteration 214, loss = nan
Iteration 215, loss = nan
Iteration 216, loss = nan
Iteration 217, loss = nan
Iteration 218, loss = nan
Iteration 219, loss = nan
Iteration 220, loss = nan
Iteration 221, loss = nan
Iteration 222, loss = nan
Iteration 223, loss = nan
Iteration 224, loss = nan
Iteration 225, loss = nan
Iteration 226, loss = nan
Iteration 227, loss = nan
Iteration 228, loss = nan
Iteration 229, loss = nan
Iteration 230, loss = nan
Iteration 231, loss = nan
Iteration 232, loss = nan
Iteration 233, loss = nan
Iteration 234, loss = nan
Iteration 235, loss = nan
Iteration 236, loss = nan
Iteration 237, loss = nan
Iteration 238, loss = nan
Iteration 239, loss = nan
Iteration 240, loss = nan
Iteration 24

...

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 6, loss = 121246466.58808777
Iteration 7, loss = 95039928.06609374
Iteration 8, loss = 104131360.08154003
Iteration 9, loss = 97300087.03777242
Iteration 10, loss = 97209169.42298026
Iteration 11, loss = 96049045.66745925
Iteration 12, loss = 96301935.99970256
Iteration 13, loss = 95725466.22506957
Iteration 14, loss = 95713486.65548320
Iteration 15, loss = 96539980.65607223
Iteration 16, loss = 98486682.50732394
Iteration 17, loss = 96148091.75138484
Iteration 18, loss = 95973277.75521567
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 1538807691.62633944
Iteration 2, loss = 1538740359.87983656
Iteration 3, loss = 1538672900.75454807
Iteration 4, loss = 1538610404.07492113
Iteration 5, loss = 1538557732.73679781
Iteration 6, loss = 1538513159.17384791
Iteration 7, loss = 1538472209.51454401
Iteration 8, loss = 1538432249.84022307
Iteration 9, loss = 1538393576.92935777
Iteration 10, loss = 1538354502.87636566
Iter

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Iteration 1, loss = 1381810906.63407159
Iteration 2, loss = 158420881.98993313
Iteration 3, loss = 340488183.20215708
Iteration 4, loss = 224009969.70918116
Iteration 5, loss = 179535415.84570125
Iteration 6, loss = 212980521.67373016
Iteration 7, loss = 218553307.53169701
Iteration 8, loss = 237982381.84157589
Iteration 9, loss = 254017672.78538030
Iteration 10, loss = 265007691.30392390
Iteration 11, loss = 275747822.89975119
Iteration 12, loss = 283754670.97095942
Iteration 13, loss = 291115054.55295187
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 1538812153.03450418
Iteration 2, loss = 1538744582.66195893
Iteration 3, loss = 1538678927.69131899
Iteration 4, loss = 1538618188.37514615
Iteration 5, loss = 1538567599.36787891
Iteration 6, loss = 1538525334.78795338
Iteration 7, loss = 1538486435.43180585
Iteration 8, loss = 1538448907.82629967
Iteration 9, loss = 1538411209.92970347
Iteration 10, loss = 1538373937.011133

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [45]:
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

#regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

best_alpha_ridge = ridge_metrics_df['MSE'].idxmin()
best_alpha_lasso = lasso_metrics_df['MSE'].idxmin()

# Regressão Ridge
ridge_model = Ridge(alpha=best_alpha_ridge)
ridge_model.fit(X_train, y_train)

# Regressão Lasso
lasso_model = Lasso(alpha=best_alpha_lasso)
lasso_model.fit(X_train, y_train)

best_params_gb = gb_metrics_df['MSE'].idxmin()

best_n_estimators, best_max_depth = best_params_gb

# Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth)
gb_model.fit(X_train, y_train)

best_params_xgb = xgb_metrics_df['MSE'].idxmin()

best_n_estimators, best_learning_rate = best_params_xgb

# XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=best_n_estimators, learning_rate=best_learning_rate)
xgb_model.fit(X_train, y_train)

best_params_nn = nn_metrics_df['MSE'].idxmin()

best_hidden_layer_sizes, best_activation = best_params_nn

# MLP Regressor
nn_model = MLPRegressor(hidden_layer_sizes=best_hidden_layer_sizes, activation=best_activation, max_iter=500)
nn_model.fit(X_train, y_train)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.523e+08, tolerance: 4.801e+06
  model = cd_fast.enet_coordinate_descent(


MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500)

In [55]:
last_row = df_features.iloc[-1]

new_data = [[last_row['lag_1'], last_row['lag_7'], last_row['ma_7']]]

tomorrow_price = model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')
tomorrow_price = ridge_model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')
tomorrow_price = lasso_model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')
tomorrow_price = gb_model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')
tomorrow_price = xgb_model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')
tomorrow_price = nn_model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')


Previsão do preço do Bitcoin para amanhã: 60368.45
Previsão do preço do Bitcoin para amanhã: 60368.45
Previsão do preço do Bitcoin para amanhã: 60368.85
Previsão do preço do Bitcoin para amanhã: 60444.01
Previsão do preço do Bitcoin para amanhã: 60446.72
Previsão do preço do Bitcoin para amanhã: 963.30


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have val